<a href="https://colab.research.google.com/github/MaralAminpour/ML-BME-Course-UofA-Fall-2023/blob/main/Week-2-Introduction-to-machine-learning/2.6-Training-models-in-scikit-learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Training models in Scikit-Learn

In this notebook, we will show how we train models in Scikit-Learn using an example of polynomial regression. We will learn how to:

* Split the data into training set and test set
* Fit a polynomial model to the training data
* Set up a pipeline to automate processing and fitting
* Use cross-validation
* Use grid search to optimize the polynomial degree hyperparameter
* Evaluate performance on the test set

In [ ]:
# This code will download the required data files from GitHub

import requests, os

# Download data from GitHub
def download_data(source, dest):
    base_url = 'https://raw.githubusercontent.com/'
    owner = 'SirTurtle'
    repo = 'ML-BME-UofA-data'
    branch = 'main'
    url = '{}/{}/{}/{}/{}'.format(base_url, owner, repo, branch, source)
    r = requests.get(url)
    f = open(dest, 'wb')
    f.write(r.content)
    f.close()

import os
if not os.path.exists('temp'):
   os.makedirs('temp')

## Generate the data

We will generate the training data from a quadratic model $y=2x^2+4x+5$ by generating random features from uniform distribution for the interval $[-3,3]$ using `rng = np.random.default_rng()` and `rng.random` and adding Gaussian noise with $\sigma=3$ to the target values using `rng.standard_normal`.

Run the code below to generate your dataset. Note that
* Feature matrix is given in the variable `X`
* Target vector is given in variable `y`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Define the true model
def TrueModel(x):
    return 2*x**2 + 4*x + 5

# Generate random samples
# To keep the random samples the same at every run we will initialize the random seed to a fixed number
rng = np.random.default_rng(seed=40)

# Generate 100 random samples
n = 100
X = 6*rng.random(n) - 3

# Generate noise for each sample
noise = 3*rng.standard_normal(n)

# Generate noisy samples
y = TrueModel(X) + noise

# Convert features X into a 2D array
X = X.reshape(-1, 1)

__Activity 1:__ Write code to print out the dimensions of the feature matrix X and target vector y. How many features and samples we have?

In [ ]:
# Print dimensions of feature matrix and target vector
print('Feature Matrix:', None) # Edit this line
print('Target Vector:', None) # Edit this line

# Print number of features and samples
print('Samples:', None) # Edit this line
print('Features:', None) # Edit this line

__Activity 2:__ Plot the generated samples. *Hint:* use `plt.plot`

In [ ]:
# Add your code here

## Split the data to training and test set

Scikit-learn offers a function `sklearn.model_selection.train_test_split` to perform the splitting of the dataset. A common pattern is to keep 80% of the data for training and use 20% for testing. This can be set using the option `test_size`. Splitting is random, so if we wish to keep the same split at every run we can use parameter `random_state`.

Finally, to create a representative test set, we can use the `stratify` parameter. In this example, we will split the data into 7 bins by rounding the feature values, and stratify the test set to have the same proportion of the data from each bin as the training set.

__Activity 3:__ Play with parameter `test_size` to see different splits

In [ ]:
from sklearn.model_selection import train_test_split

# Create bins for the target values
bins = np.round(X)

# Perform stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=42)

# Plot the training and test data
plt.plot(X_train, y_train, 'o', label='Training set')
plt.plot(X_test, y_test, 'o', label='Test set')
plt.legend()
plt.title('Train test split')
plt.xlabel('Feature')
plt.ylabel('Target value')
plt.show()

## Fitting a polynomial model

Scikit-learn does not implement a polynomial regression model. Instead, the polynomial regression is performed in two steps:
1. Polynomial feature transformation - a __transformer__ object `PolynomialFeatures` transforms each feature $x$ into a feature vector $(1,x,x^2,...,x^M)$, where polynomial degree $M$ is defined by setting parameter `degree`
2. Performing multivariate linear regression - a __regressor__ object `LinearRegression` fits the model
$y=x_0+x_1w_1+...+x_Mw_M$ to the data.

Note that if we combine these two steps, we will have $x_j=x^j$ and therefore create a polynomial model $y=x_0+w_1x+...+w_Mx^M$

The code below demonstrates how we can fit the 2nd order polynomial model to the training set and evaluate it on the test set.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Create polynomial features object of second degree
M = 2
poly_features = PolynomialFeatures(degree=M)

# Transform training features (note conversion to 2D array)
X_train_poly = poly_features.fit_transform(X_train)

# Create and fit multivariate linear regression model
# We do not need intercept, because the first feature is 1
model = LinearRegression(fit_intercept=False)
model.fit(X_train_poly, y_train)

# Evaluate the model on the test set
X_test_poly = poly_features.fit_transform(X_test)
r2_test = model.score(X_test_poly, y_test)
print('R2 score on test set is', round(r2_test, 2))

### Plot the fitted polynomial model

__Activity 4:__ Fill in the code below to plot the second order polynomial model that we just fitted. Then change the polynomial degree in the cell above, and rerun both cells to see how the model changes.

In [ ]:
# Plot the training and test data
# Add your code here

# Define the feature space
X_curve=np.linspace(-3, 3, 100).reshape(-1, 1)

# Perform the polynomial feature transformation of X_curve
X_curve_poly = poly_features.transform(X_curve)

# Predict target values using X_curve_poly
y_curve = model.predict(X_curve_poly)

# Plot the curve
# Add your code here

# Annotate the plot
plt.title('Polynomial fit $M={}$'.format(M))
plt.xlabel('Feature')
plt.ylabel('Target value')
plt.show()

__Activity 4:__ Print the coefficients of the model

In [ ]:
# Show the coefficients of the model
print(model.coef_)
print('{2:.1f}x^2 + {1:.1f}x + {0:.1f} '.format(model.coef_[0], model.coef_[1], model.coef_[2]))

We see that we have recovered a model $y=2.1x^2 + 4.0x + 5.6$ which is fairly close to our true model.

Okay, we have fit a polynomial model to our data. So far so good!

## Setting up a model pipeline

It is not always convenient to perform several steps for model training, prediction and evaluation. In particular, if the model becomes an input to another `sklearn` object, such as hyperparameter search using cross-validation that we will introduce below, the steps need to be unified in a single object.

Scikit-learn implements a class `sklearn.pipeline.Pipeline` to join multiple __steps__ into one model. `Pipeline` implements methods `fit`, `predict`, `score` and others to offer a unified API with other sklearn objects.

In the code below we will demonstrate how to join the polynomial feature transformation and multivariate linear regression into a single model.

In [ ]:
from sklearn.pipeline import Pipeline

# Polynomial regression model is created as a 'pipeline'
# combining creation of features (1,x,x^2,...) followed
# by multivariate linear regression
pipeline = Pipeline((
("poly_features", PolynomialFeatures(degree=2)),
("lin_reg", LinearRegression(fit_intercept=False)) ))

# Fit the model using features and labels
pipeline.fit(X_train, y_train)

# Calculate performance
r2_test = pipeline.score(X_test, y_test)
print('R2 score on test set is', round(r2_test, 2))

Let's look in detail at the syntax for creating the pipelines. It is in the form `Pipeline(steps)`, where the input argument `steps` is a list of transforms and models to be chained, in the order with which they need to be called. Each step is described by a tuple `(name, model)`, where `name` is a string chosen by the user and `model` is a `sklearn` object.

To access the original objects joint in a `Pipeline` we use attribute `Pipeline.named_steps`. For example, the linear regression object defined in the cell above can be accessed as:

In [ ]:
# Access LinearRegression object
pipeline.named_steps["lin_reg"]

## Cross-validation

Even if we stratify our test set, it is not guaranteed that it is representative and the performance may vary every time we create a different split. A more robust technique is to perform cross-validation, where we split the data into $k$ groups (folds), and each fold will be used to measure performance exactly once, while remaining data are used to fit the model. Average performance over the $k$ folds will be much more robust. If there are no hyperparameters to tune, cross-validation can be used directly to measure the performance of the model.

We will use the [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function.

In scikit-learn cross validation is called using `cross_val_score(estimator, X, y)` with arguments
* `estimator:` the model to be fit
* `scoring:` the score to use; if this parameter is not specified, use the model's default score
* `X:` the feature matrix
* `y:` optional target values or labels for supervised models
- `cv:` optional argument that defines the number of folds.

The model returns an array with scores from each fold. Note that the function cross_val_score is only for measuring the scores that the model would produce on the $k$ folds. It does not return the trained models. For that task, use GridSearchCV, as described below.

In [ ]:
from sklearn.model_selection import cross_val_score

# Define the model
pipeline = Pipeline((
("poly_features", PolynomialFeatures(degree=2)),
("lin_reg", LinearRegression(fit_intercept=False)),))

# Perform 5-fold cross-validation
scores = cross_val_score(pipeline, X, y, cv=5)

# Print scores
print('Scores:', np.around(scores, 2))
print('Cross-validated R2 score: mean = {}, standard deviation = {}'.format(round(scores.mean(),3),round(scores.std(),3)))

## Hyperparameter search using grid search

Finally we will show how we can automatically find the optimal hyperparameters in scikit-learn. One of the most common ways is to train a model for each hyper-parameter value and measure its performance using cross-validation. This approach is implemented in `sklearn.model_selection.GridSearchCV`.

First we need to create the grid search object using `GridSearchCV(estimator, param_grid)` with arguments:
* `estimator:` the model to be fitted
* `param_grid:` a dictionary, with parameter names as keys, and parameter values as lists
* `cv:` optional parameter to determine the number of folds

We will now find the optimal polynomial degree automatically, using the `GridSearchCV`. First, we need to define our parameter grid dictionary. Because we have a `Pipeline` object, the parameter name is composed of the step name `"poly_features"` and parameter name `"degree"` which are joined by two underscores `__`.

In [ ]:
# Propose a range of possible values for the polynomial degree
degrees = range(1,20)

# Define parameter dictionary
# Because the model is a pipeline, the name of the parameter is composed
# of the name of the step and parameters names joint by a double underscore
parameters = {"poly_features__degree": degrees}

Next we create the `GridSearchCV` object and run the hyperparameter optimization using method `fit`, which is performed on the training set. The chosen polynomial model can be accessed using the variable `best_estimator_`

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create grid search object
grid_search = GridSearchCV(pipeline, parameters, cv=5)

# Perform the search over all hyperparameter values
grid_search.fit(X_train, y_train)

# This is the chosen model
best_model = grid_search.best_estimator_

We can also access the chosen polynomial, and the best R2 score identified through cross-validation.

In [ ]:
# Degree of the best model
best_degree = best_model.named_steps['poly_features'].degree
print('The best polynomial degree is', best_degree)

# Best score
best_score = best_model.score(X_train, y_train)
grid_search.best_score_
print('The best cross-validated R2 score on training set is', round(best_score, 2))

## Evaluate performance on test set

We can evaluate the performance on the test set directly using a trained `GridSearchCV` by calling its method `score`. It will return the score of the model with the best hyperparameter setting identified during the search and fitted to the whole training set.

In [ ]:
r2 = grid_search.score(X_test, y_test)
print('R2 score on test set is:', round(r2, 2))

## Exercise

Now we will load the data 'noisy_data.csv' using `pandas`. The file contains two columns:
* column zero contains features `X`
* column one contains target values `y`

Run the code below to create `X` and `y` and plot the data.

In [ ]:
# imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Download data
download_data('Week-2-Introduction-to-machine-learning/data/noisy_data.csv', 'temp/noisy_data.csv')

# Load data
import pandas as pd
df = pd.read_csv('temp/noisy_data.csv')

print(df)

X = df.values[:,0].reshape(-1,1)
y = df.values[:,1]

plt.plot(X, y, 'o')
plt.title('Noisy data')
plt.xlabel('Feature')
plt.ylabel('Target value')
plt.show()

Your task is to fit a polynomial model to the data. Perform the following steps:
* Perform stratified train-test split
* Create a polynomial model using `Pipeline`
* Use `GridSearchCV` to find the optimal polynomial degree and print out the best cross-validated score
* Calculate performance on the test set
* Plot the fitted model
* Print out the coefficients of the model

Can you work out what was the underlying polynomial model? (*Hint: the coefficients were whole numbers*)

In [ ]:
# STRATIFIED TRAIN TEST SPLIT
bins = np.round(X)
#X_train, X_test, y_train, y_test = None # Edit this line

In [ ]:
# CREATE POLYNOMIAL MODEL
pipeline = None # Edit this line

In [ ]:
# CREATE PARAMETER GRID
#parameters = {"poly_features__degree": range(1,20)}
parameters = None # Edit this line
# Tip: Using 20 for max degree sometimes overfits the data!

In [ ]:
# OPTIMISE HYPERPARAMETERS
# Create grid search object
grid_search = None # Edit this line

# Fit GridSearchCV object
# Add your code here

# Extract the selected model
best_model = None # Edit this line

# Print out the best score and chosen polynomial degree
#print('The best cross-validated R2 score on training set is',round(grid_search.best_score_,2))
#print('The best polynomial degree is',best_model.named_steps["poly_features"].degree)

In [ ]:
# CALCULATE PERFORMANCE ON TEST SET
r2 = None # Edit this line
#print('R2 score on test set is', round(r2,2))

In [ ]:
# PLOT THE FITTED MODEL
x_curve = None # Edit this line
y_curve = None # Edit this line

# Plot the training data
# Add your code here

# Plot the test data
# Add your code here

# Plot the model curve

plt.title('Polynomial fit')
plt.xlabel('Feature')
plt.ylabel('Target value')
plt.show()

In [ ]:
# COEFFICIENTS OF THE MODEL
coef = None # Edit this line
#print('Coefficients: ', np.around(coef))

The original model is $y=2-3x+x^3$

*Note: Splitting of training and test set is random, so different solutions are possible. From the shape of the dataset, we can judge that the underlying polynomial was cubic. If you obtain a polynomial of a higher degree, try to re-run your code until you get a cubic polynomial. That will help you to work out true model coefficients.*